# Adding SmartShots to a shots table


## 1. Introduction

### What are SmartShots?

SmartShots are curated collections of expert-written KQL queries paired with natural language descriptions and vector embeddings.  
They serve as a knowledge base of operational intelligence, capturing proven query patterns and domain expertise in a searchable, reusable format that LLMs can leverage through semantic retrieval.  
This notebook will outline the process of adding a selection of shots, and will include the flow of Explanation - Embedding - Ingestion.

### Why SmartShots Matter

- **Enhance LLM Query Generation**: Provide high-quality, contextually relevant KQL examples to improve AI-assisted query creation  
- **Accelerate Problem Solving**: Enable instant retrieval of battle-tested queries through semantic search  
- **Preserve Institutional Knowledge**: Capture and maintain critical operational queries and patterns  
- **Democratize Expertise**: Make expert query patterns accessible across all skill levels  

### Integration with RTI MCP

SmartShots integrate with the Fabric RTI MCP ([Docs](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/mcp-overview) - [Repo](https://github.com/microsoft/fabric-rti-mcp)) through the `get_shots` tool, which performs semantic search across your SmartShots collection.  
When users describe their analytical needs in natural language, the tool retrieves the most relevant expert queries by comparing the user's request embedding against stored query embeddings using cosine similarity.

> ℹ️ This notebook assumes the user has working knowledge of KQL.  
It does not cover query syntax, optimization techniques, or basic Kusto fundamentals.  
Users unfamiliar with KQL may wish to consult the official documentation: https://learn.microsoft.com/en-us/azure/data-explorer/kusto/query  

> ⚠️ Generating natural language explanations and embeddings for large query sets can be both expensive and time-consuming.  
Each API call to AI plugins incurs costs and processing time, which can accumulate quickly. This notebook demonstrates the process with a small example set (10–20 queries).  
For production deployments with hundreds or thousands of queries, plan for extended processing times and budget accordingly.


---

## 2. Notebook Setup

### Prerequisites

This notebook requires:

- Notebook set to PySpark (Python) and not Python
- Access to a Fabric cluster and database
- Enabled callout policy for AI plugins: `ai_chat_completion_prompt` and `ai_embeddings` (required if the plugins will be used)
- AI plugin endpoints for `ai_chat_completion_prompt` and `ai_embeddings` (required if the plugins will be used)
- [Kqlmagic](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/notebook-kql-magic) extension - See install block below

For more info regarding the AI plugins see:
- https://learn.microsoft.com/en-us/kusto/query/ai-chat-completion-plugin
- https://learn.microsoft.com/en-us/kusto/query/ai-embeddings-plugin

In [ ]:
import numpy as np
import pandas as pd
import os
from io import StringIO

###### **If you need to install Kqlmagic, unfreeze and run the code block below.**

In [ ]:
# %pip install Kqlmagic

In [ ]:
%env KQLMAGIC_LOAD_MODE=silent
# %env KQLMAGIC_LOG_LEVEL=DEBUG
%reload_ext Kqlmagic
%config Kqlmagic.auto_dataframe = True

---


## 3. Environment Setup

### Fabric Connection

Connect to the Fabric environment where the SmartShots table will be created and maintained.

Set the following variables:

- `SHOTS_CLUSTER`: Fabric cluster URI
- `SHOTS_DATABASE`: Database name for SmartShots
- `SHOTS_TABLE`: Table name (e.g., `shots`)  
- `CHAT_COMPLETION_ENDPOINT`: Endpoint for `ai_chat_completion_prompt` plugin
- `EMBEDDING_ENDPOINT`: Endpoint for `ai_embeddings` plugin (Ensure the embedding model matches the model you use in the Fabric RTI MCP configuration)
- `CSV_PATH`: Path to your pre-curated CSV Shots file (optional, see Shot Selection section below)


In [ ]:
SHOTS_CLUSTER = "your-fabric-cluster-uri"
SHOTS_DATABASE= "your-database-name"
SHOTS_TABLE = "your-shots-table-name"
CHAT_COMPLETION_ENDPOINT = 'https://your-chat-completion-endpoint.openai.azure.com/openai/deployments/preferred-chat-model/chat/completions?api-version=2025-01-01-preview;impersonate'
EMBEDDING_ENDPOINT = 'https://your-embedding-endpoint.openai.azure.com/openai/deployments/preferred-embedding-model/embeddings?api-version=2025-01-01-preview;impersonate'


---

### Connect to database and verify connection

In [ ]:
%kql kusto://code;cluster=SHOTS_CLUSTER;database=SHOTS_DATABASE
%kql .show tables | take 10


### Enable AI Plugins

If you are planning to use the AI kql plugins you should first enable the callout policy.
For more info see:

- https://learn.microsoft.com/en-us/kusto/query/ai-chat-completion-prompt-plugin
- https://learn.microsoft.com/en-us/kusto/query/ai-embeddings-plugin

---

## 4. Creating a Shots Table

### Recommended Minimal Schema

| Column              | Type        | Description                                |
|-------------        |----------   |--------------------------------------------|
| `Timestamp`         | datetime    | Creation/update time                       |
| `NaturalLanguage`   | string      | Natural language description               |
| `Query`             | string      | KQL query                                  |
| `EmbeddingVector`   | dynamic     | Vector embedding array                     |
| `Author`            | string      | Query author                               |
| `Tags`              | dynamic     | Array of category tags                     |

### Best Practices

- Add necessary columns to match your needs  
- Use a dedicated table with access controls  
- Include metadata for traceability  
- Partition by domain/team if needed 
- Set Row Level Security if needed (for more info see: [Row level security policy](https://learn.microsoft.com/en-us/kusto/management/row-level-security-policy?view=microsoft-fabric))


You can use this kql command, and adapt it to your needs to create the shots table.  
If you would like to create the table from within the notebook make sure to unfreeze the code block.

In [ ]:
%kql .create table {SHOTS_TABLE} (Timestamp:datetime, NaturalLanguage:string, Query:string, EmbeddingVector:dynamic, Author:string, Tags:dynamic)

---

## 5. Selecting / Cherry-Picking Shots

### Methods

- Manual selection of shots 
- Using `.show queries` to find used queries. See [show queries command](https://learn.microsoft.com/en-us/kusto/management/show-queries-command) for more info  

\
This notebook will demonstrate the process with a set of pre-curated queries set below.  
- If you are using a csv file of selected shots you may choose to first ingest the file to a temporary table in your database.  
- If you are using a `.show queries` kql you may simply project the query text and pass the results to the next steps.  

Regardless, you are free to adapt the queries provided in this notebook to suit your needs.

### Best Practices

- Prioritize expert-written, frequently used, operationally critical queries.  
- Avoid overly basic/trivial queries. They add noise and don’t help in this case.
- Consider using clustering methods if using the `.show queries` method to reduce large query logs into representative patterns. This helps identify recurring query structures and operational themes. 


In [ ]:
comma_separated_sample_shots = '''
```StormEvents
| where EventType has "Thunderstorm"
| summarize count() by bin(StartTime, 7d)```,
```StormEvents
| join kind = inner (PopulationData | top 1 by Population desc) on State
| distinct Source
| count```,
```US_States
| project State=features.properties.NAME, Area=geo_polygon_area(features.geometry)
| top 2 by Area desc
| summarize arg_min(Area, *)```,
```StormEvents
| summarize countEvents=count() by State
| join kind=inner (
    PopulationData
) on State
| order by countEvents desc
| project State, Population
| take 1```,
```StormEvents
| where State endswith "o"
| partition by State
    (
        summarize TotalDeaths = sum(DeathsDirect + DeathsIndirect) by EventType, State
        | top 3 by TotalDeaths
        | where TotalDeaths > 0
    )```
'''

---

## 6. Adding a natural language explanation

### Purpose

Natural language explanations are used for similarity comparison with the user's natural language request.  
The natural language explanation is what gets embedded into a vector, and the `get_shots` tool utilizes this vector for comparison and retrieval purposes.  
This notebook will demonstrate auto-generation of natural language explanations using the `ai_chat_completion_prompt` kql plugin.

### Methods

- Manual writing  
- Auto-generation via `ai_chat_completion_prompt` - method used in this notebook
- Auto-generation using your preferred method/endpoint.  

### Tips

- Focus on user intent  
- Avoid technical phrasing  
- Do not include usage context like "when/why someone would use this query"  

✅ Examples:
- "Find services with elevated error rates in the last hour"  
- "Show me the top 10 users by data consumption this month"  
- "Identify unusual login patterns from new geographic locations"  

❌ Avoid:
- "This query joins tables and calculates error percentages"  
- "Aggregate AuthLogs by UserID and GeoIP with anomaly detection"  


---

## 7. Embedding

### Purpose

The embedding step converts the natural language explanation into a high-dimensional vector that captures its semantic meaning. 
These embeddings are used to compare user requests against stored SmartShots using cosine similarity. 
This enables the `get_shots` tool to retrieve the most relevant queries based on meaning rather than keywords. 

### Methods

- Use `ai_embeddings_plugin` to convert Natural Language text into vector embeddings - method used in this notebook.
- Use any other embedding method you prefer.

Reference: https://learn.microsoft.com/en-us/kusto/query/ai-embeddings-plugin

### Compatibility

Ensure the embedding model matches your Fabric RTI MCP configuration.  
Example models:

- `text-embedding-ada-002`  
- `text-embedding-3-small`  
- `text-embedding-3-large`  

### Performance

- API rate limits apply  
- Each embedding call adds ~0.5–2 seconds  
- Batch processing recommended  
- Monitor token usage  


---

## 8. Ingestion

### Purpose
The ingestion step stores each SmartShot into the designated Fabric table so it can be retrieved later via semantic search.  
This includes the original query, its natural language explanation, the embedding vector, and relevant metadata.  
Once ingested, these records become part of the searchable corpus used by the `get_shots` tool in Fabric RTI MCP.

### Append SmartShots to Shots Table

Include:

- Query  
- Natural language explanation  
- Embedding  
- Metadata (author, tags, timestamp)  

### Ensure

- Valid KQL syntax  
- Non-empty NL text  
- Correct embedding dimensions  
- Metadata populated  

---

## 9. Run Query

The query below will generate a natural language explanation (utilizing the `NL_PROMPT` provided below) for each shot, generate an embedding, and finally ingest the data in to the shots table (according to the minimal schema provided above).  
Once again, feel free to adjust the query to fit your needs and changes.

In [ ]:
NL_PROMPT = '''Generate a natural language request that would lead a user to write the following KQL query.
The request should reflect what the user wants to know or investigate, phrased naturally and without technical jargon.
The output should strictly be the generated natural language. After you generate the natural language make sure it matches the results given by the KQL query.
KQL query: '''

explain_embed_ingest_query = f'''
.append {SHOTS_TABLE} <|
let chatCompletionEndpoint="{CHAT_COMPLETION_ENDPOINT}";
let embeddingEndpoint="{EMBEDDING_ENDPOINT}";
let embeddingModel = tostring(extract("deployments/([^/]*)/", 1, embeddingEndpoint));
let prompt = ```{NL_PROMPT}```;
let Shots = datatable(Query:string) [{comma_separated_sample_shots}];
Shots
| extend NLprompt = strcat(prompt, Query)
| evaluate ai_chat_completion_prompt(NLprompt, chatCompletionEndpoint)
| project-rename NaturalLanguage = NLprompt_chat_completion
| evaluate ai_embeddings(NaturalLanguage, embeddingEndpoint)
| project-rename EmbeddingVector = NaturalLanguage_embeddings
| project Timestamp = now(), NaturalLanguage, Query, EmbeddingVector, Author = "SmartShotsNotebook", Tags = pack_array(embeddingModel)
'''

In [ ]:
# Run query to generate natural language explanations, vector embeddings, and ingest into shots table
%kql -query explain_embed_ingest_query

> ⚠️ This query will fail if the table does not already exist.  
In that case you can go back to section 4 and create the table according to your preferences.

###### We can now sample a few of the latest shots in the table and see that our ingestion worked:

In [ ]:
%kql {SHOTS_TABLE} | top 5 by Timestamp

---

## 10. Additional Notes

> ℹ️ This notebook does not implement duplicate detection or batching logic for large sets of SmartShots.  
These aspects - such as deduplication, batching embeddings, or ingestion operations - are left to the user's discretion based on their scale, performance, and operational needs.

### Permissions

- Encourage open access  
- Use row-level security for sensitive queries  

### Maintenance

- Monthly reviews  
- Remove deprecated shots  
- Refresh embeddings if model changes  
- Archive outdated versions  

### Resources

- https://github.com/microsoft/fabric-rti-mcp  
- https://learn.microsoft.com/en-us/azure/data-explorer/kusto/query/  
- https://learn.microsoft.com/en-us/fabric/real-time-analytics/overview  

### Next Steps

- Test semantic retrieval  
- Validate `get_shots` integration  
- Collect feedback from your team  
- Scale to larger sets  
- Automate quality monitoring  
- Build contribution pipeline  


---